## Definición de la Target
*Your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly*

In [168]:
ruta_abs = r'./data/train.csv'

## Librerías
En la celda abajo indicada se mostrarám todas las librerías empleadas durante el proceso.

In [169]:
import pandas as pd
import numpy as np


In [170]:
df = pd.read_csv(ruta_abs)

In [171]:
print(15*"-","Columnas del DataFrame de train:",15*"-")
col_names = df.columns
print(col_names)

print(15*"-","Chequeo duplicados:",15*"-")
print(df.duplicated().any())


print(15*"-","Número total de nulos:",15*"-")
df.isna().sum()

--------------- Columnas del DataFrame de train: ---------------
Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')
--------------- Chequeo duplicados: ---------------
False
--------------- Número total de nulos: ---------------


PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

De aqui se pueden obtener diferentes conclusiones, como se observa la fuente de datos no presenta ningún duplicado, sin embargo en cuanto a nulos se necesita una evaluación más exhaustiva, dado que la muesttra con la que se cuenta es solo de ocho mil IDs.

## Evaluacion de Nulos

In [172]:
def eval_Nan(df,col_eval,col_based):
    print(15*"-","{} NANs:".format(col_eval),15*"-")
    print(len(df[col_eval].unique()))
    print(df[col_eval].unique())
    print(df.groupby([col_eval],as_index=False).count().sort_values(["PassengerId"],ascending=False)[[col_eval,col_based]])
    df.loc[df[col_eval].isna()==True].count()

In [173]:
col = col_names.to_list().copy()
col.remove("PassengerId")

for i in col:
    eval_Nan(df,i,"PassengerId")


--------------- HomePlanet NANs: ---------------
4
['Europa' 'Earth' 'Mars' nan]
  HomePlanet  PassengerId
0      Earth         4602
1     Europa         2131
2       Mars         1759
--------------- CryoSleep NANs: ---------------
3
[False True nan]
   CryoSleep  PassengerId
0      False         5439
1       True         3037
--------------- Cabin NANs: ---------------
6561
['B/0/P' 'F/0/S' 'A/0/S' ... 'G/1499/S' 'G/1500/S' 'E/608/S']
         Cabin  PassengerId
6156   G/734/S            8
764     C/21/P            7
5092  G/1368/P            7
571     B/82/S            7
6527   G/981/S            7
...        ...          ...
2553  F/1245/P            1
2552  F/1244/S            1
2551  F/1243/S            1
2550  F/1243/P            1
6559     T/3/P            1

[6560 rows x 2 columns]
--------------- Destination NANs: ---------------
4
['TRAPPIST-1e' 'PSO J318.5-22' '55 Cancri e' nan]
     Destination  PassengerId
2    TRAPPIST-1e         5915
0    55 Cancri e         1800
1  PSO

To DOs
    1. Mirar los registros donde los CryoSleep son True para ver si han realizado algún tipo de gasto extra.
    2. Evaluar aquellos registros donde los nombre de los viajeros son iguales, ¿han realizado varios viajes?

In [174]:
df.loc[(df["RoomService"].isna()==True) & (df["CryoSleep"]==True),["RoomService"]]

,RoomService
25,NaN
83,NaN
233,NaN
400,NaN
889,NaN
...,...
8312,NaN
8361,NaN
8380,NaN
8412,NaN


In [179]:
bill_Amount = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]

for i in bill_Amount:
    df.loc[(df["CryoSleep"]==True) & df[i].isna()==True]=0